In [7]:
import pickle
from tqdm import tqdm
import numpy as np
import time

X_RF_path = 'C:\\Users\\Carrt\\OneDrive\\Code\\Motion Privacy\\Attacking Models\\RF Attack Model\\X.pkl'
X_SGN_path = 'C:\\Users\\Carrt\\OneDrive\\Code\\Motion Privacy\\Attacking Models\\SGN Attack Model\\data\\X.pkl'

In [8]:
with open('average-skeleton-joints.pkl', 'rb') as f:
    average_skeleton_joints = pickle.load(f)

def rad_to_deg(rad): return rad * 180 / np.pi

paths_to_end_effectors = [
    [0,1,20,2,3],
    [0,1,20,8,9,10,11,23],
    [0,1,20,8,9,10,11,24],
    [0,1,20,4,5,6,7,21],
    [0,1,20,4,5,6,7,22],
    [0,12,13,14,15],
    [0,16,17,18,19]
]

In [71]:

def normalize_skeleton(skeleton, format='rf', verbose=False):
    if format == 'rf':
        new_skele = [0 for i in range(25 * 3)]
        new_skele[0:3] = skeleton[0:3]
        for path in paths_to_end_effectors:
            for connection in range(len(path) - 1):
                # If second joint in connection is already calculated, skip
                if new_skele[path[connection + 1] * 3] != 0:
                    continue

                vector = skeleton[path[connection] * 3: path[connection] * 3 + 3] - skeleton[path[connection + 1] * 3: path[connection + 1] * 3 + 3]
                length = np.linalg.norm(vector)
                normalized_vector = vector / length
                if verbose: print(f"{path[connection]}-{path[connection + 1]}: {length} - {normalized_vector}")

                x_rotation = np.arctan2(normalized_vector[1], normalized_vector[2])
                y_rotation = np.arctan2(normalized_vector[0], normalized_vector[2])
                z_rotation = np.arctan2(normalized_vector[0], normalized_vector[1])

                if verbose:
                    print(f"Radians: X: {x_rotation} Y: {y_rotation} Z: {z_rotation}")
                    print(f"Degrees: X: {rad_to_deg(x_rotation)} Y: {rad_to_deg(y_rotation)} Z: {rad_to_deg(z_rotation)}")

                distance = average_skeleton_joints[f"{path[connection]}-{path[connection + 1]}"]
                if verbose: print(f"Distance: {distance}")

                dx = distance * np.cos(y_rotation) * np.cos(z_rotation)
                dy = distance * np.sin(z_rotation) * np.cos(x_rotation)
                dz = distance * np.sin(x_rotation) * np.cos(y_rotation)
                if verbose: print(f"dx: {dx} dy: {dy} dz: {dz}")

                new_pos = skeleton[path[connection] * 3: path[connection] * 3 + 3] + np.array([dx, dy, dz])
                if verbose: print(f"New Position: {new_pos}")
                new_skele[path[connection + 1] * 3: path[connection + 1] * 3 + 3] = new_pos

    elif format == 'sgn':
        new_skele = np.zeros((300,150))
        i = 0
        for frame in skeleton:
            skele1 = frame[0:75]
            skele2 = frame[75:150]
            
            if not skele1.any(): new_skele1 = np.zeros(75)
            else: new_skele1 = normalize_skeleton(skele1, format='rf').flatten()
            if not skele2.any(): new_skele2 = np.zeros(75)
            else: new_skele2 = normalize_skeleton(skele2, format='rf').flatten()

            new_skeleton = np.concatenate((new_skele1, new_skele2))
            new_skele[i] = new_skeleton
            i += 1
    return np.array(new_skele)

In [ ]:
# For Random Forest
# (75)

with (open(X_RF_path, "rb")) as f:
    X = pickle.load(f)
    
X_ = {}

for key in tqdm(X.keys()):
    X_[key] = []
    for skeleton in X[key]:
        skeleton = normalize_skeleton(skeleton, format='rf')
        if np.isnan(skeleton).any(): continue
        X_[key].append(skeleton)
    X_[key] = np.array(X_[key])

with open('X_RF_normalized.pkl', 'wb') as f:
    pickle.dump(X_, f)

In [18]:
with (open(X_SGN_path, "rb")) as f:
    X = pickle.load(f)

In [74]:
# For SGN
# (300, 150)

X_ = {}

for key in tqdm(X.keys(), desc='Actor', position=0, leave=True):
    X_[key] = []
    for video in tqdm(X[key], desc='Video', position=1, leave=False, miniters=20):
        skeleton = normalize_skeleton(video, format='sgn')
        if np.isnan(skeleton).any(): continue
        X_[key].append(skeleton)
    X_[key] = np.array(X_[key])

with open('X_SGN_normalized.pkl', 'wb') as f:
    pickle.dump(X_, f)


Actor:   0%|          | 0/106 [00:00<?, ?it/s]C:\Users\Carrt\AppData\Local\Temp\ipykernel_14888\623741319.py:13: RuntimeWarning: invalid value encountered in divide
  normalized_vector = vector / length
Actor: 100%|██████████| 106/106 [1:06:11<00:00, 37.46s/it]
